In [17]:
import os
import pandas as pd

# Veri seti dizini (dosyaların bulunduğu yer)
data_dir = "./KETI/"

# Her oda için CSV dosyalarını bulma
csv_files = []
for root, _, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".csv"):
            csv_files.append(os.path.join(root, file))

# Her dosyayı okuyup işlemek için boş bir DataFrame oluştur
all_data = []

for file in csv_files:
    # Sensör tipini ve oda bilgisini dosya yolundan al
    sensor_type = os.path.basename(file).split('.')[0]
    room = os.path.basename(os.path.dirname(file))
    
    # Dosyayı oku ve kolon isimlerini düzenle
    df = pd.read_csv(file)
    df.columns = ['ts', sensor_type]
    df["room"] = room
    
    # Zaman damgasını datetime formatına çevir ve ts_min_bignt ve event_ts_min kolonlarını ekle
    df['event_ts_min'] = pd.to_datetime(df['ts'], unit='s')
    df['ts_min_bignt'] = df['ts']
    
    all_data.append(df)
    
# Her dosyanın içeriğini birleştirme
combined_df = pd.concat(all_data, axis=0)

# combined_df'in ilk birkaç satırını kontrol edelim
print(combined_df.head())

# Aynı zaman ve oda için verileri birleştirmek için pivot table kullanma
pivot_df = combined_df.pivot_table(index=['event_ts_min', 'ts_min_bignt', 'room'], 
                                   values=['co2', 'humidity', 'light', 'pir', 'temperature'],
                                   aggfunc='first').reset_index()

# Sonuçları bir CSV dosyasına kaydet
output_file = "./processed_sensor_data2.csv"
pivot_df.to_csv(output_file, index=False)

print(f"Veri işleme tamamlandı ve {output_file} dosyasına kaydedildi.")

# Kaydedilen dosyanın ilk birkaç satırını kontrol et
with open(output_file, 'r') as f:
    lines = f.readlines()
    for line in lines[:5]:  # İlk 5 satırı yazdır
        print(line)


           ts    co2 room        event_ts_min  ts_min_bignt  humidity  light  \
0  1377299108  495.0  413 2013-08-23 23:05:08    1377299108       NaN    NaN   
1  1377299110  487.0  413 2013-08-23 23:05:10    1377299110       NaN    NaN   
2  1377299127  496.0  413 2013-08-23 23:05:27    1377299127       NaN    NaN   
3  1377299130  495.0  413 2013-08-23 23:05:30    1377299130       NaN    NaN   
4  1377299134  500.0  413 2013-08-23 23:05:34    1377299134       NaN    NaN   

   pir  temperature  humidity-checkpoint  light-checkpoint  
0  NaN          NaN                  NaN               NaN  
1  NaN          NaN                  NaN               NaN  
2  NaN          NaN                  NaN               NaN  
3  NaN          NaN                  NaN               NaN  
4  NaN          NaN                  NaN               NaN  
Veri işleme tamamlandı ve ./processed_sensor_data2.csv dosyasına kaydedildi.
event_ts_min,ts_min_bignt,room,co2,humidity,light,pir,temperature

2013-08-2

In [20]:
import pandas as pd

# İşlenmiş veriyi yükle
processed_file = "./processed_sensor_data2.csv"
df = pd.read_csv(processed_file)

# Eksik değerleri içeren satırları çıkarma
df = df.dropna(subset=['co2', 'humidity', 'light', 'pir', 'temperature'])

# Sensör değerlerinin tümü 0 olan satırları çıkarma
columns_to_check = ['co2', 'humidity', 'light', 'pir', 'temperature']
df_filtered = df[(df[columns_to_check] != 0).any(axis=1)]

# Filtrelenmiş veri setini kaydetme
filtered_file = "./cleaned_sensor_data2.csv"
df_filtered.to_csv(filtered_file, index=False)

print(f"Temizlenmiş veri seti {filtered_file} dosyasına kaydedildi.")


Temizlenmiş veri seti ./cleaned_sensor_data2.csv dosyasına kaydedildi.
